### Check connect

In [ ]:
from pathlib import Path

# from cpuinfo import get_cpu_info  # py-cpuinfo
import hashlib
from cryptocode import encrypt, decrypt
from webdav3.client import Client  # webdavclient3
from webdav3.exceptions import WebDavException  # webdavclient3
import json
import platform


def generate_key_word():
    cpu_inf = str(
        str(platform.system())
        + str(platform.node())
        + str(platform.version())
        + str(platform.processor())
        + str(platform.architecture())
        + str(platform.machine())
    ).replace(" ", "")
    d = hashlib.sha256()
    d.update(cpu_inf.encode("utf-8", errors="ignore"))
    # with open(
    #     Path(__file__).parent.resolve() / "settings2.ini", "w", encoding="utf-8"
    # ) as file:
    #     file.write(d.hexdigest())
    return d.hexdigest()


def encrypt_word(word: str, key_word: str) -> str:
    return encrypt(key_word)


def decrypt_word(word: str, key_word: str) -> str:
    return decrypt(word, key_word)


def crypt_input():
    # sm = MDApp.get_running_app().root
    # key_word = "The best password for Clean_up_the_Tasks"
    # login = sm.ids.settings_screen.ids.login_field.text
    # password = sm.ids.settings_screen.ids.password_container.ids.password_field.text

    key_word = generate_key_word()
    login = encrypt("dvo.2103@mail.ru", key_word)
    password = encrypt(" ", key_word)

    data = {
        "webdav_hostname": "https://webdav.cloud.mail.ru",
        "webdav_login": login,
        "webdav_password": password,
    }

    settings_json = Path(__file__).parents[1] / "data/settings.json"
    with open(settings_json, "w", encoding="utf-8") as file:
        json.dump(data, file)


# crypt_input()


def check_client():
    settings_json = Path(__file__).parents[1] / "data/settings.json"
    with open(settings_json, "r", encoding="utf-8") as file:
        data = json.load(file)

    key_word = generate_key_word()
    decrypt_data = {
        "webdav_login": decrypt(data["webdav_login"], key_word),
        "webdav_password": decrypt(data["webdav_password"], key_word),
    }
    data.update(decrypt_data)

    # login = decrypt(credentials["webdav_login"], key_word)
    # password = decrypt(credentials["webdav_password"], key_word)
    # data = {
    #     "webdav_hostname": credentials["webdav_hostname"],
    #     "webdav_login": login,
    #     "webdav_password": password,
    # }
    try:
        client = Client(data)
        my_files = client.list()
        print(my_files)
    except WebDavException as exception:
        print(exception)


check_client()


def json_dump():
    data = {
        "webdav_hostname": "https://webdav.cloud.mail.ru",
        "webdav_login": "super_login",
        "webdav_password": "super_password",
    }
    settings_json = Path(__file__).parents[1] / "data/settings.json"

    with open(settings_json, "w", encoding="utf-8") as file:
        json.dump(data, file)


# json_dump()


def json_load():
    settings_json = Path(__file__).parents[1] / "data/settings.json"
    with open(settings_json, "r", encoding="utf-8") as file:
        data = json.load(file)

    print(data["webdav_login"], "\n:::\n", data["webdav_password"])


# json_load()


def cpu_information():
    cpu_inf = (
        "%s%s%s%s"
        % (
            get_cpu_info()["model"],
            get_cpu_info()["family"],
            "".join(get_cpu_info()["flags"]),
            get_cpu_info()["arch"],
        )
    ).replace(" ", "")

    print(cpu_inf)

### Get hardware id

In [ ]:
import cpuinfo  # py-cpuinfo
import hashlib


def get_hardware_id():
    """
    Get an unique hardware ID, based on CPU info. All the times called, the value will be the same.
    >>> get_hardware_id()
    'f8ef57d64aae6f3c45200b39di422bd6ca625d9a79655cb3aa6e171ef6f93013aa16c2df2f2b0359dfaf1782ba6fda94300506cdd9b21fdaf1264fbd0e47abb89'
    :return: string with the hardware ID
    :rtype: str
    """
    _ci = cpuinfo.get_cpu_info()

    cpu_inf = (
        "%s%s%s%s" % (_ci["model"], _ci["family"], "".join(_ci["flags"]), _ci["arch"])
    ).replace(" ", "")

    # print(cpu_inf)
    d = hashlib.sha512()
    d.update(cpu_inf.encode("utf-8", errors="ignore"))

    return d.hexdigest()


# get_hardware_id()
# print(cpuinfo.get_cpu_info())
print(get_hardware_id())

### Mail.ru tools

In [ ]:
"""
Набор функций для работы с облаком.
Для работы требует установки библиотеки:
pip install webdavclient3 cryptocode
"""

from pathlib import Path

from cryptocode import decrypt
from webdav3.client import Client


def decrypt_word(word: str, psw: str) -> str:
    """
    Дешифрование слова, переданного в функцию.
    """
    return decrypt(word, psw)


def authorize(psw: str) -> (Client, bool):
    """
    Авторизация в облаке. Декодирование логина и пароля.
    Создание клиента с авторизацией.
    Возвращение клиента из функции для использования в других функциях.
    """
    datas = [x.strip() for x in open("setting.ini", "r", encoding="utf-8") if x.strip()]
    login = decrypt_word(datas[0], psw)
    password = decrypt_word(datas[1], psw)
    if login:
        data = {
            "webdav_hostname": "https://webdav.cloud.mail.ru",
            "webdav_login": login,
            "webdav_password": password,
        }

        client = Client(data)
        return client
    return False


def check_setting() -> bool:
    """
    Проверка существования файла настроек.
    """
    if not Path("setting.ini").exists():
        return False
    return True


def connect_cloud_folder(client: Client, path: str) -> dict:
    """
    Читает содержимое переданной директории в облаке.
    Возвращает словарь с полученными значениями.
    """
    my_files = client.list(path, get_info=True)
    return my_files


def create_cloud_folder(client, path: str) -> bool:
    """
    Создание директории в облаке. Возвращает True,
    если создание директории прошло успешно.
    """
    return client.mkdir(path)


def delete_cloud_folder(client: Client, path: str):
    """
    Удаление переданной директории в облаке.
    """
    client.clean(path)


def upload_cloud_file(client: Client, remote_path: str, local_path: str):
    """
    Загрузка файла в облако.
    """
    client.upload_sync(remote_path, local_path)


def download_cloud_file(client: Client, remote_path: str, local_path: str) -> bool:
    """
    Скачивание файла из облака.
    """
    client.download_sync(remote_path, local_path)
    return True


def move_cloud_object(client: Client, remote_path_from: str, remote_path_to: str):
    """
    Перемещение файла в облаке.
    """
    client.move(remote_path_from, remote_path_to)


def copy_cloud_object(client: Client, remote_path_from: str, remote_path_to: str):
    """
    Копирование файла в облаке.
    """
    client.copy(remote_path_from, remote_path_to)

### Switch screen

example

In [ ]:
from kivy.uix.screenmanager import Screen
from kivy.lang import Builder
from kivy.properties import NumericProperty
from kivy.animation import Animation

from kivymd.app import MDApp

Builder.load_file("screens/titlescreen.kv")


class TitleScreen(Screen):
    col_offset = NumericProperty(0)

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        anim = Animation(col_offset=1)
        anim += Animation(col_offset=0)

        anim.repeat = True
        anim.start(self)

    def on_touch_down(self, touch):
        if touch.is_double_tap:
            self.change_screen()

    def change_screen(self):
        app_root = MDApp.get_running_app().root
        setattr(app_root, "current", "_main_screen_")